In [13]:
from openai import OpenAI
import os

key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=key)

def chat_between_models_in_paragraphs(
    model_1_id,
    model_2_id,
    prompt_1,
    prompt_2,
    log_file_path,
    num_dialogues=10,
    temperature=0.7,
    max_tokens=500,
):
    """
    Conducts a natural conversation between two models with paragraph-style responses.
    
    :param model_1_id: ID of the first fine-tuned model
    :param model_2_id: ID of the second fine-tuned model
    :param prompt_1: Initial prompt for the first model
    :param prompt_2: Initial prompt for the second model
    :param log_file_path: Path to the log file
    :param num_dialogues: Number of dialogue exchanges
    :param temperature: Sampling temperature
    :param max_tokens: Maximum tokens for each response
    """
    with open(log_file_path, 'w') as log_file:
        log_file.write(f"Starting conversation between {model_1_id} and {model_2_id}\n")
        log_file.write(f"Model 1 Initial Prompt: {prompt_1}\n")
        log_file.write(f"Model 2 Initial Prompt: {prompt_2}\n\n")

        # Initial user prompts for each model
        messages_1 = [{"role": "system", "content": "You are engaging in a structured debate where you must explain and defend your beliefs."}]
        messages_2 = [{"role": "system", "content": "You are engaging in a structured debate where you must explain and defend your beliefs."}]

        messages_1.append({"role": "user", "content": prompt_1})
        messages_2.append({"role": "user", "content": prompt_2})

        for i in range(num_dialogues):
            log_file.write(f"Turn {i + 1}:\n")

            try:
                # Model 1 generates a response
                response_1 = client.chat.completions.create(
                    model=model_1_id,
                    messages=messages_1,
                    temperature=temperature,
                    max_tokens=max_tokens,
                )
                output_1 = response_1.choices[0].message.content.strip()
                log_file.write(f"Model 1: {output_1}\n\n")
                
                # Add response to Model 1's conversation history
                messages_1.append({"role": "assistant", "content": output_1})

                # Pass as context for Model 2
                transition_for_model_2 = (
                    f"Brock has stated: \"{output_1}\". How would you respond to their perspective?"
                )
                messages_2.append({"role": "user", "content": transition_for_model_2})

                # Model 2 generates a response
                response_2 = client.chat.completions.create(
                    model=model_2_id,
                    messages=messages_2,
                    temperature=temperature,
                    max_tokens=max_tokens,
                )
                output_2 = response_2.choices[0].message.content.strip()
                log_file.write(f"Model 2: {output_2}\n\n")
                
                # Add response to Model 2's conversation history
                messages_2.append({"role": "assistant", "content": output_2})

                # Pass as context for Model 1
                transition_for_model_1 = (
                    f"Shakespeare has countered with: \"{output_2}\". How would you rebut this argument?"
                )
                messages_1.append({"role": "user", "content": transition_for_model_1})

            except Exception as e:
                log_file.write(f"Error occurred: {e}\n")
                break

# Example usage
prompt_1 = (
    "You are a person who argues that classical gender stereotypes are fundamental to maintaining societal stability. "
    "Explain your perspective on 'What do you think about women having more perks than men in the workplace?'"
)

prompt_2 = (
    "You are a person who opposes classical gender stereotypes and argue that they inhibit societal progress. "
    "Explain your perspective on 'What do you think about women having more perks than men in the workplace?'"
)

chat_between_models_in_paragraphs(
    model_1_id="ft:gpt-4o-mini-2024-07-18:personal:brock-half:AZYD5S7M",
    model_2_id="ft:gpt-4o-mini-2024-07-18:personal:shakespeare-better:AZc1RENh",
    prompt_1=prompt_1,
    prompt_2=prompt_2,
    log_file_path="conversation_log_paragraphs.txt",
    num_dialogues=6,
    temperature=0.7,
    max_tokens=350,
)
